#### Let's define the problem statement for this project: <br> 

We are tasked by the State University to evaluate the attractivity of the town for future University staff relocating with their family. We will look at what the average cost of a house is, and **what are the key parameters when buying one**.<br>  

Sources :<br>https://edition.cnn.com/2022/02/07/success/us-housing-market-affordability-and-inventory-feseries/index.html<br>
*Also hurt are homebuyers earning between $75,000 and $100,000. This group, the report found, can afford a maximum home price of $433,480. Researchers determined these prices by making some assumptions, including that the buyer is not spending more than 30% of their income on housing and that the purchase is financed with a 30-year fixed-rate mortgage*

Sources: https://www.payscale.com/research/US/Location=Ames-IA/Salary

##### To achieve this, we will run a linear regression to predict the housing prices in Ames, and we will narrow down on the 3 bedrooms, and eventually compare it with parents earnings and cost of living in Ames 

##### The aim is to publish these findings on a website to manage expectations for buyewrs on the hunt and provide transparency beyond the usual median price 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import RidgeCV,Lasso, LassoCV

In [2]:
# import our dataset
df = pd.read_csv('./output/df_eda_complete.csv')

In [3]:
df.shape

(2051, 69)

In [4]:
#we've realized that the process of exporting our df created an index"Unammed:0" column, let's drop it now 

In [5]:
df.drop(columns=['Unnamed: 0'],inplace=True)

#### Let's prepare the model for linear regression
1. One-hot encore categorical variables
2. Train-test-split
3. standardize data
4. run Linear regression + Lasso + Run Ridge | Cross validate along the way | provide the best r2
5. Select best model
6. Check coefficients and interpret the model
8. answer the problem statement

#### 1.Lets OHE categorical variables 

In [6]:
# Dummify all categorical columns - pd.get_dummies is the pandas option to do one-hot encoding
X = df.drop(columns = ["SalePrice"]) # get a subset_df 

In [7]:
y = df["SalePrice"]

In [8]:
X.shape, y.shape #checking our split

((2051, 67), (2051,))

In [9]:
#identifying categorical columns
cat_cols = X.select_dtypes("object").columns

In [10]:
#OHE dummies
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

In [11]:
X.shape, y.shape #quick check

((2051, 224), (2051,))

In [12]:
X.shape

(2051, 224)

### 2 . Train-test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape # checking outputs

((1640, 224), (411, 224), (1640,), (411,))

#### 3. Standardizing data

In [15]:
#we will use StandardScaler to apply scaling to our entire dataset
ss = StandardScaler()
X_train = ss.fit_transform(X_train) # fit_transform X_train only, industry practice 
X_test = ss.transform(X_test)

In [16]:
#from the course, making sure our steps yields expected outputs
print(f'Z_train shape is: {X_train.shape}')
print(f'y_train shape is: {y_train.shape}')
print(f'Z_test shape is: {X_test.shape}')
print(f'y_test shape is: {y_test.shape}')

Z_train shape is: (1640, 224)
y_train shape is: (1640,)
Z_test shape is: (411, 224)
y_test shape is: (411,)


#### 4.run Linear regression, Lasso and ridge. Cross validate along the way. select best model 

In [17]:
#let's start with linear regression and see how our model performs
#using the help of https://www.pluralsight.com/guides/linear-lasso-ridge-regression-scikit-learn to navigate
lr = LinearRegression()

In [18]:
lr.fit(X_train,y_train)

LinearRegression()

In [19]:
#Our model is created. let's use it to generate predictions and check their accuracy using MSE and R2

In [20]:
pred_train_lr = lr.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_lr)))
print(r2_score(y_train, pred_train_lr))

19982.09257701591
0.9362521325498426


The model fit on the training set seems to performs really well. let's check out on the test set

In [21]:
pred_test_lr= lr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_lr))) 
print(r2_score(y_test, pred_test_lr))

7858805382214672.0
-9.786231048532444e+21


**Baseline score** is is bad. Our model cannot generalize to new(unseen) data. means it is overfitted. let's shrinks this model using regularization

In [22]:
cross_val_score(lr, X_train, y_train).mean()

-1.7488137616592164e+25

In [23]:
#baseline
print(" OLS ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 OLs on test is {lr.score(X_test, y_test)}')
print(f'RMSE OLS on test is {np.sqrt(mean_squared_error(y_test,pred_test_lr))}') 
print(f'MAE OLS on test is {metrics.mean_absolute_error(y_test, pred_test_lr)}')

====== OLS =======
R2 OLs on test is -9.786231048532444e+21
RMSE OLS on test is 7858805382214672.0
MAE OLS on test is 392728063868679.3


our cross_val_score is terrible too. We know something is wrong with this model

#### On to Lasso<br>
We want our model to be able to predict future data accurately, so we need to reduce the variance in our model

In [24]:
#by using LASSO we apply an additional penalty term (alpha) to irrelevant features
#we search for alphas in logspace (between 10^-2 = 0.01 and 10^0 = 1)
l_alphas = np.logspace(-3, 1, 10)

In [25]:
# Cross-validate over our list of Lasso alphas.
lasso_model = LassoCV(alphas=l_alphas, cv=5, max_iter=5000)

In [26]:
lasso_model.fit(X_train,y_train)

c:\Users\galvi\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1213242765.7184448, tolerance: 805335324.204708
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\galvi\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2045290906.2124023, tolerance: 805335324.204708
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\galvi\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2098055619.1104126, tolerance: 805335324.204708
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\galvi\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: Co

LassoCV(alphas=array([1.00000000e-03, 2.78255940e-03, 7.74263683e-03, 2.15443469e-02,
       5.99484250e-02, 1.66810054e-01, 4.64158883e-01, 1.29154967e+00,
       3.59381366e+00, 1.00000000e+01]),
        cv=5, max_iter=5000)

In [27]:
#checking model with the best alpha found in the step above
lasso_best = Lasso(alpha =lasso_model.alpha_).fit(X_train, y_train)

In [28]:
#checking our model results
print(lasso_best.score(X_train, y_train))
print(lasso_best.score(X_test, y_test))

0.936128683073466
0.8620892944358395


Looks better; lets check :
1. which alpha was selected to be the best to improve our model
3. what are the performance metrics for the test set

In [29]:
lasso_model.alpha_ # checking which alpha was selected

10.0

In [30]:
pred_test_lasso= lasso_best.predict(X_test)

In [31]:
#baseline
print(" OLS ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {lr.score(X_test, y_test)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_lr))}') 
print(f'MAE on test is {metrics.mean_absolute_error(y_test, pred_test_lr)}')

print("LASSO ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {r2_score(y_test,pred_test_lasso)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_lasso))}') 
print(f'MAE on test is {metrics.mean_absolute_error(y_test,pred_test_lasso)}')

====== OLS =======
R2 on test is -9.786231048532444e+21
RMSE on test is 7858805382214672.0
MAE on test is 392728063868679.3
======LASSO ======
R2 on test is 0.8620892944358395
RMSE on test is 29501.750891602034
MAE on test is 17601.691221587917


It seems that our model  generalizes well to new data. 

#### On to Ridge

In [32]:
#instantiating the model
ridge_cv = RidgeCV()

In [33]:
r_alphas = np.logspace(0, 5, 100)

# Cross-validate over our list of ridge alphas.
# alphas: pass an Array of alpha values to try. It is still the Regularization strength
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2', cv=5).fit(X_train, y_train)

In [34]:
pred_train_rr = ridge_cv.predict(X_train) #establishing predictions for the training set
print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))#evaluating metric #1
print(r2_score(y_train, pred_train_rr))#evaluating metric #2

pred_test_rr= ridge_cv.predict(X_test)#establishing predictions for the test set

24272.038051756972
0.9059418968688339


In [35]:
#baseline
print(" OLS ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {lr.score(X_test, y_test)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_lr))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test, pred_test_lr),3)}')

print("LASSO ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {r2_score(y_test,pred_test_lasso)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_lasso))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test,pred_test_lasso),3)}')

print("RIDGE ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {r2_score(y_test, pred_test_rr)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_rr))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test,pred_test_rr),3)}')


====== OLS =======
R2 on test is -9.786231048532444e+21
RMSE on test is 7858805382214672.0
MAE on test is 392728063868679.3
======LASSO ======
R2 on test is 0.8620892944358395
RMSE on test is 29501.750891602034
MAE on test is 17601.691
======RIDGE ======
R2 on test is 0.8670153672562568
RMSE on test is 28970.06906158316
MAE on test is 18373.69


we are  getting better values  with ridge, however the MAE is  lower with Lasso

#### On to Elastic net

In [36]:
from sklearn.linear_model import ElasticNetCV

# Set up a list of alphas to check.
enet_alphas = np.linspace(0.5, 1.0, 100)# Return evenly spaced numbers over a specified interval

# Instantiate model.
enet_model = ElasticNetCV(alphas=enet_alphas, cv=5) #l1_ratiofloat, default=0.5

In [37]:
# Fit model using optimal alpha.
enet_model = enet_model.fit(X_train, y_train)

In [38]:
# Generate predictions.
enet_model_preds_train = enet_model.predict(X_train)
enet_model_preds = enet_model.predict(X_test)


# Evaluate model.
print(enet_model.score(X_train, y_train))
print(enet_model.score(X_test, y_test))

0.9012777775588424
0.8661264142981273


In [39]:
#baseline
print(" OLS ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {lr.score(X_test, y_test)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_lr))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test, pred_test_lr),3)}')

print("LASSO ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {r2_score(y_test,pred_test_lasso)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_lasso))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test,pred_test_lasso),3)}')

print("RIDGE ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {r2_score(y_test, pred_test_rr)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,pred_test_rr))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test,pred_test_rr),3)}')

print("ENET ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(f'R2 on test is {r2_score(y_test, enet_model_preds)}')
print(f'RMSE on test is {np.sqrt(mean_squared_error(y_test,enet_model_preds))}') 
print(f'MAE on test is {round(metrics.mean_absolute_error(y_test,enet_model_preds),3)}')

====== OLS =======
R2 on test is -9.786231048532444e+21
RMSE on test is 7858805382214672.0
MAE on test is 392728063868679.3
======LASSO ======
R2 on test is 0.8620892944358395
RMSE on test is 29501.750891602034
MAE on test is 17601.691
======RIDGE ======
R2 on test is 0.8670153672562568
RMSE on test is 28970.06906158316
MAE on test is 18373.69
======ENET =======
R2 on test is 0.8661264142981273
RMSE on test is 29066.734871547895
MAE on test is 18591.85


# We conclude that RIDGE provides the best regularization for our model: largest r2 lowest MEA/ RMSE

In [40]:
# lets visualize our model

In [41]:
#using this visualization from Kobe solution lab
final_ridge_df = pd.DataFrame({'variable':X.columns,
                            'coef':ridge_cv.coef_,
                            'abs_coef':np.abs(ridge_cv.coef_)})
final_ridge_df.sort_values('coef', ascending=False)[:10]

,variable,coef,abs_coef
3,Overall Qual,9775.276887,9775.276887
73,Neighborhood_NridgHt,7741.695656,7741.695656
15,Gr Liv Area,7569.144949,7569.144949
79,Neighborhood_StoneBr,6055.512959,6055.512959
12,1st Flr SF,5185.305128,5185.305128
23,Garage Cars,4852.245070,4852.245070
11,Total Bsmt SF,4358.559418,4358.559418
13,2nd Flr SF,4109.740328,4109.740328
24,Garage Area,3977.091312,3977.091312
7,Mas Vnr Area,3930.947738,3930.947738


In [42]:
print ('Percent variables zeroed out:', np.sum((ridge_cv.coef_ == 0))/float(X.shape[0])) # checking the penalty imposed from enet

Percent variables zeroed out: 0.00048756704046806434


In [43]:
print(ridge_cv.intercept_)

180257.36829268292


### Given the table above, we can answer our problem statement

We are tasked by the State University to evaluate the attractivity of the town for future University staff relocating with their family. We will look at what the average cost of a house is, and what are the key parameters to consider if you're single, with kids and with cars.

The key drivers of the price houses in Ames are Overall Quality, Gr Liv Area, Total Bsmt SF, 1st Floor, NridgHt Neighborhood, StoneBr and Garage Area.

<br>

Let's interprets the first 3: <br>
1. The intercept is 181k, which is the average sale price. that's bizarre. We expect a minimum price but not as high as the average of the dataset
the top 3 price predictors are Overall Quality,Gr Liv area, and whether or not the house is in NridgHt 

So we would recommend the university to 
1. look at tier-2 quality houses 
2. highlight the "cheaper" neighborhood in our model
3. downplay the ownership of a garage (it does drive the price up with a very positive beta). Encourage staff to cycle to work 